In [1]:
import sys
sys.path.append('../HeapModel')

In [2]:
from copy import deepcopy
import angr
from angr import SimStatePlugin, SimProcedure
from mstate import HeapState

In [3]:
class HeapPlugin(SimStatePlugin):
    def __init__(self, h=None):
        super().__init__()
        if(h is None):
            self.heap_state = HeapState(0)
        else:
            self.heap_state = h
        
    def set_heap(self, h):
        self.heap_state = h
    
    @SimStatePlugin.memo
    def copy(self, memo):
        heap_copy = deepcopy(self.heap_state)
        return HeapPlugin(heap_copy)

In [4]:
b = angr.Project('TestCases/UAF')
ss = b.factory.blank_state()

WARNING | 2020-06-18 20:00:06,185 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


In [5]:
ss.register_plugin('heap', HeapPlugin())

In [6]:
class Malloc(SimProcedure):
    i = 0
    def run(self):
        print(f'malloc called {self.i}')
        self.i+=1

class Free(SimProcedure):
    i = 0
    def run(self):
        print(f'free called {self.i}')
        self.i+=1        

In [7]:
b.hook_symbol('malloc', Malloc())
b.hook_symbol('free', Free())

WARNING | 2020-06-18 20:00:07,005 | angr.project | Address is already hooked, during hook(0x797070, <SimProcedure Malloc>). Re-hooking.
WARNING | 2020-06-18 20:00:07,006 | angr.project | Address is already hooked, during hook(0x797950, <SimProcedure Free>). Re-hooking.


7960912

In [8]:
sm = b.factory.simulation_manager(ss)
sm.step()
print(sm.active[0].heap)

WARNING | 2020-06-18 20:00:07,060 | angr.state_plugins.symbolic_memory | The program is accessing memory or registers with an unspecified value. This could indicate unwanted behavior.
WARNING | 2020-06-18 20:00:07,061 | angr.state_plugins.symbolic_memory | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING | 2020-06-18 20:00:07,061 | angr.state_plugins.symbolic_memory | 1) setting a value to the initial state
WARNING | 2020-06-18 20:00:07,062 | angr.state_plugins.symbolic_memory | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING | 2020-06-18 20:00:07,062 | angr.state_plugins.symbolic_memory | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY_REGISTERS}, to suppress these messages.
WARNING | 2020-06-18 20:00:07,063 | angr.state_plugins.symbolic_memory | Filling register rdx with 8 unconstrained bytes referenced from 0x400580 (_start+0x0 in UA